### Todo:
- Fix scrapper exception bug
- Loop through users if fail

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import subprocess
import names
from selenium.webdriver.common.by import By
import math
import random
import pandas as pd
import os
import json
import re
import sys
import shutil
from datetime import datetime

In [2]:
def scrap_profile(name,count):
    try:
        command = "instagram-scraper " + name + " -u maskradiogreece@gmail.com -p code12345 -t image -m " + str(count) + " --media-metadata"
        subprocess.check_output(command)
    except subprocess.CalledProcessError as e:
        print(f"Exception occured scrapping profile of user {name}")
        raise Exception("Sorry, papala")

In [3]:
def evaluate_profile(folder_name):
    print("Opening file")
    file = open(f'./{folder_name}/{folder_name}.json', encoding="utf8")
    print("Loading json")
    data = json.load(file)
    pattern = re.compile(r'\d+_\d+_\d+_n.jpg')
    images = []
    likes = []
    for item in data["GraphImages"]:
        finds = re.search(pattern,item["display_url"])
        # If the post is at least 2 days old then add it to data or else skip it
        if (datetime.today() - datetime.fromtimestamp(item["taken_at_timestamp"])).days > 2:
            images.append(finds[0])
            likes.append(item["edge_media_preview_like"]["count"])
    maxLikes = max(likes)
    if maxLikes < 1000:
        print(f"Not enough likes deleting {folder_name}")
        file.close()
        print("Removing Folder")
        shutil.rmtree(rf"./{folder_name}/") #removes the folder.
        print("Returning")
        return False
    for index,name in enumerate(images):
        try:
            os.rename(rf"./{folder_name}/{name}",rf"./{folder_name}/{round(likes[index]/maxLikes,2):.2f} {folder_name}{name}")
        except:
            pass
    print("Closing File")
    file.close()
    print("Moving Folder")
    shutil.move(f"./{folder_name}/",f"./Images/{folder_name}/")
    return True

### Import Chrome Driver for Selenium

In [4]:
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")

In [5]:
driver.maximize_window()

In [6]:
driver.get("https://www.instagram.com/")

### To continue make sure that:
- You logged in with your credentials
- Accepted cookies
- Then run code below

In [ ]:
num_images = 0
throttle = 60
while True:
    print("Going to start page")
    driver.get("https://www.instagram.com/")
    inputs = driver.find_elements_by_tag_name("input")
    # The 3rd one is the search 
    search = inputs[2]
    random_female_name = names.get_first_name(gender='female') 
    search.send_keys(random_female_name)
    time.sleep(4)
    # Get List of Users
    list = driver.find_elements(By.XPATH, '//div[@role="none"]')
    print("Length of list is",len(list))
    # Create List 
    profile_links = []
    for user in list:
        link = user.find_element_by_tag_name("a")
        link = link.get_attribute("href")
        if link.find("explore") == -1:
            profile_links.append(link)
    searched_links = pd.read_csv("./Links.csv")
    searched_links = searched_links["Links"].to_list()
    for link in profile_links:
        if link in searched_links:
            continue
        driver.get(link);
        time.sleep(random.randint(1,5))
        try:
            is_private = driver.find_element(By.XPATH, "//*[text()[contains(., 'Αυτός ο λογαριασμός είναι ιδιωτικός')]]" )
            continue 
        except: 
            name = link.split("/")[3]
            post = driver.find_element(By.XPATH, "//*[text()[contains(., 'δημοσιεύσεις') or contains(., 'δημοσίευση')]]" )
            post.find_element_by_tag_name("span")
            num_posts = int(post.find_element_by_tag_name("span").get_attribute("innerHTML").replace(",",""))
            if num_posts < 10:
                continue
            get_num_posts = math.floor(num_posts * 0.5)
            get_num_posts = min(math.floor(num_posts * 0.5),200)
            print(f"Getting {get_num_posts} from {name}")
            try:
                scrap_profile(name,get_num_posts)
                print(f"Evaluating images on {name} folder")
                added = evaluate_profile(name)
                if added:
                    num_images += get_num_posts
                    print(f"Got {num_images} in total")
            except:
                print(f"There was error on getting {name}")
                # Sleep bettwen throttle to throttle + 2 minutes
                # We get throttling from API
                sleep_time = random.randint(throttle, throttle + 60)
                print(f"Going to sleep for {sleep_time}s")
                time.sleep(sleep_time)
                # Increase throttle by 1 min
                throttle += 60
                # Append Failed Link to end of array to try and fetch again 
                profile_links.append(link)
                continue
            searched_links.append(link)
            df = pd.DataFrame(searched_links,columns=["Links"])
            df.to_csv('./Links.csv',index=False)

Going to start page
Length of list is 56
Getting 51 from darlenejanesummers
Evaluating images on darlenejanesummers folder
Opening file
There was error on getting darlenejanesummers
Going to sleep for 66s
Getting 18 from darlene_sid_official
Evaluating images on darlene_sid_official folder
Opening file
There was error on getting darlene_sid_official
Going to sleep for 133s
Getting 173 from darlene_goodman
Evaluating images on darlene_goodman folder
Opening file
There was error on getting darlene_goodman
Going to sleep for 232s
